# Collect shapes and generate relations

Here we these this package's ability to collect shapes from OpenStreetMap
and to use them as fodder for generating prescribed pairwise relations.
We generate and plot a few "true" and a few "false" examples for each of 
several types of relations.


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx
import shapely
import pyproj

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

In [ ]:
from geo_relations import OSMShapeCollector
from geo_relations import RelationGenerator
from geo_relations import draw_shape

## Setup

In [ ]:
# Define a lon/lat center and a box size from which to pull shapes.
center_lat, center_lon = 43.134101, -70.926430
extent = 10000.0 # meters; this represents both width and height

In [ ]:
# Get a set of Point, LineString, and Polygon objects from our sample box.
collector = OSMShapeCollector(center_lon, center_lat, extent)
shapes = collector.collect(['linestrings', 'polygons'])
shapes['type'].value_counts()

## Generate shape pairs with given relationships
For each type of relation, generate some positive and some negative cases, and plot them.

In [ ]:
aoi_width = 100
aoi_height = 100
margin = 10
cases_per = 3
relations = [
    'point-on-linestring', 'point-in-polygon', 'linestring-intersects-linestring',
    'linestring-intersects-polygon', 'polygon-intersects-polygon', 'polygon-borders-polygon'
]

for relation in relations:
    generator = RelationGenerator(shapes, bounds=[0, 0, aoi_width, aoi_height], margin=margin, scale=[0.1, 0.5])
    fig = make_subplots(2, cases_per, subplot_titles = ['true'] * cases_per + ['false'] * cases_per)
    
    for i in range(cases_per):
        a, b = generator.generate(relation, True, max_attempts=100)
        draw_shape(a, fig, irow=1, icol=i+1, name=a.geom_type, color='red')
        draw_shape(b, fig, irow=1, icol=i+1, name=b.geom_type, color='blue')
    
    for i in range(cases_per):
        a, b = generator.generate(relation, False)
        draw_shape(a, fig, irow=2, icol=i+1, name=a.geom_type, color='red')
        draw_shape(b, fig, irow=2, icol=i+1, name=b.geom_type, color='blue')
    
    fig['layout']['title'] = relation
    fig['layout']['width'] = 800
    fig['layout']['height'] = 600
    fig['layout']['showlegend'] = False
    
    for i in range(cases_per * 2):
        fig['layout']['xaxis%d' % (i+1)]['range'] = [0, aoi_width]
        fig['layout']['yaxis%d' % (i+1)]['range'] = [0, aoi_height]
    fig.show()


# fig.print_grid()